# 🎯 07번: Corrective RAG (CRAG)

## 🎯 학습 목표
- **검색 결과를 자가 평가**하는 Retrieval Evaluator
- **3가지 액션 경로** (Correct/Incorrect/Ambiguous)
- **폴백 전략**으로 시스템 견고성 향상

## 💡 핵심 아이디어

```
검색했다고 끝이 아니다. 검증하고 대비하라!

Query: "민법 제103조 내용"
  → 검색 → 평가 → score 0.85 (correct) ✅
  → Reranking → 답변

Query: "오늘 날씨 어때?"
  → 검색 → 평가 → score 0.15 (incorrect) ❌
  → 폴백 → "법률 정보가 아닙니다"
```

## 📋 실습 구성
1. Part 1: Retrieval Evaluator
2. Part 2: 3가지 액션 경로
3. 퀴즈: 액션 예측하기
4. 임계값 튜닝
5. Part 3: Naive vs CRAG 비교

In [ ]:
# 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers pandas matplotlib

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import pandas as pd
import matplotlib.pyplot as plt
import re

print("✅ 라이브러리 준비 완료!")

In [ ]:
# 06번과 동일한 법률 문서 재사용
legal_docs = [
    {
        "content": """민법 제103조 (반사회질서의 법률행위)
조문: 선량한 풍속 기타 사회질서에 위반한 사항을 내용으로 하는 법률행위는 무효로 한다.
적용: 공서양속 위반 계약, 불법 원인 급여
판례: 대법원 2015da234567 - 도박 채무 계약 무효
키워드: 공서양속, 무효, 반사회적""",
        "law": "민법 제103조",
        "code": "103",
        "category": "민법"
    },
    {
        "content": """민법 제750조 (불법행위의 내용)
조문: 고의 또는 과실로 인한 위법행위로 타인에게 손해를 가한 자는 그 손해를 배상할 책임이 있다.
적용: 교통사고, 명예훼손, 재산권 침해
판례: 대법원 2018da567890 - 의료과실 손해배상
키워드: 불법행위, 손해배상, 고의과실""",
        "law": "민법 제750조",
        "code": "750",
        "category": "민법"
    },
    {
        "content": """형법 제257조 (상해죄)
조문: 사람의 신체를 상해한 자는 7년 이하의 징역, 10년 이하의 자격정지 또는 1천만원 이하의 벌금에 처한다.
적용: 폭행으로 인한 신체 손상
판례: 대법원 2019do12345 - 상해죄 성립 요건
키워드: 상해, 신체, 폭행""",
        "law": "형법 제257조",
        "code": "257",
        "category": "형법"
    },
    {
        "content": """형법 제329조 (절도)
조문: 타인의 재물을 절취한 자는 6년 이하의 징역 또는 1천만원 이하의 벌금에 처한다.
적용: 재물 절도, 소매치기
판례: 대법원 2017do98765 - 절도죄 기수시기
키워드: 절도, 재물, 절취""",
        "law": "형법 제329조",
        "code": "329",
        "category": "형법"
    },
    {
        "content": """상법 제15조 (상호등록)
조문: 상인은 그 상호를 등기하여야 한다. 등기한 상호는 이를 양도할 수 있다.
적용: 상호 등록, 상호권 보호
판례: 대법원 2016da11111 - 상호 부정사용 금지
키워드: 상호, 등록, 상호권""",
        "law": "상법 제15조",
        "code": "15",
        "category": "상법"
    },
    # Confuser 문서들 (관련도 낮은 문서)
    {
        "content": """민법 제1조 (법원)
조문: 민사에 관하여 법률에 규정이 없으면 관습법에 의하고, 관습법이 없으면 조리에 의한다.
적용: 법원, 법률 해석의 기준
키워드: 법원, 관습법, 조리""",
        "law": "민법 제1조",
        "code": "1",
        "category": "민법"
    },
    {
        "content": """형법 제1조 (범죄의 성립과 처벌)
조문: 범죄의 성립과 처벌은 행위시의 법률에 의한다.
적용: 죄형법정주의, 소급입법 금지
키워드: 죄형법정주의, 법률, 처벌""",
        "law": "형법 제1조",
        "code": "1",
        "category": "형법"
    },
    {
        "content": """상법 제1조 (상사에 관한 규정의 해석)
조문: 상사에 관하여 본법에 규정이 없으면 상관습법에 의하고 상관습법이 없으면 민법의 규정에 의한다.
적용: 상법 해석 기준
키워드: 상사, 상관습법, 민법""",
        "law": "상법 제1조",
        "code": "1",
        "category": "상법"
    },
    {
        "content": """헌법 제1조 (대한민국의 국체와 주권)
조문: 대한민국은 민주공화국이다. 대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.
적용: 민주주의, 국민주권
키워드: 민주공화국, 주권, 국민""",
        "law": "헌법 제1조",
        "code": "1",
        "category": "헌법"
    },
    {
        "content": """민사소송법 제1조 (소송의 목적)
조문: 민사소송은 당사자의 권리의 적정한 실현을 목적으로 한다.
적용: 소송 목적, 권리 실현
키워드: 민사소송, 권리, 목적""",
        "law": "민사소송법 제1조",
        "code": "1",
        "category": "민사소송법"
    },
    {
        "content": """형사소송법 제1조 (목적)
조문: 형사소송은 형사실체법의 적정한 실현과 적법절차의 보장을 목적으로 한다.
적용: 형사절차, 적법절차
키워드: 형사소송, 적법절차, 실체법""",
        "law": "형사소송법 제1조",
        "code": "1",
        "category": "형사소송법"
    }
]

# Document 객체 생성
documents = [Document(
    page_content=doc["content"],
    metadata={"law": doc["law"], "category": doc["category"]}
) for doc in legal_docs]

# 임베딩 모델 + 벡터스토어 구축
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents, embeddings)

# 법률용 토크나이저 (키워드 매칭용)
def legal_tokenizer(text):
    """법률 문서용 토크나이저 - 키워드 매칭에 사용"""
    text = re.sub(r'(제\d+조)', r' \1 ', text)
    text = re.sub(r'(\d{4}[다도가나]\d+)', r' \1 ', text)
    text = re.sub(r'\b(\d+)\b', r' \1 ', text)
    tokens = [t for t in text.split() if t.strip()]
    return tokens

print("✅ 법률 문서 벡터스토어 구축 완료!")
print(f"   문서 수: {len(documents)}개")
print(f"   카테고리: 민법, 형법, 상법, 헌법, 소송법")
print()
print("📋 핵심 문서:")
for doc in legal_docs[:5]:
    print(f"   - {doc['law']} ({doc['category']})")

## 🔍 Part 1: Retrieval Evaluator - 검색 결과 평가

### 핵심 질문
**"검색한 문서가 질문에 답하기에 적절한가?"**

### Evaluator의 역할
1. 검색된 문서의 **relevance(관련도)** 점수 계산
2. 점수를 바탕으로 **액션 결정** (Correct/Incorrect/Ambiguous)

### 평가 방법 2가지

#### 방법 1: 키워드 기반 (간단, 빠름)
```python
# 질문과 문서의 키워드 겹침 비율 계산
overlap = len(query_words & doc_words) / len(query_words)
```

#### 방법 2: LLM 기반 (정확, 느림)
```python
# LLM에게 관련도 평가 요청
"이 문서가 질문에 적절한가요? 0.0~1.0 점수로:"
```

### 실습: 키워드 기반 Evaluator 구현

In [ ]:
# Retrieval Evaluator 구현

def calculate_relevance_keyword(query, doc):
    """
    키워드 기반 relevance 계산
    
    방법: Query 키워드가 문서에 얼마나 포함되어 있는가?
    """
    # 토큰화
    query_tokens = set(legal_tokenizer(query))
    doc_tokens = set(legal_tokenizer(doc.page_content))
    
    # Jaccard similarity
    if len(query_tokens) == 0:
        return 0.0
    
    intersection = query_tokens & doc_tokens
    score = len(intersection) / len(query_tokens)
    
    return score

def evaluate_retrieval(query, docs, method="keyword"):
    """
    검색 결과 평가 및 액션 결정
    
    Returns:
        max_score: 최고 relevance 점수 (0.0 ~ 1.0)
        action: "correct" / "incorrect" / "ambiguous"
        individual_scores: 각 문서별 점수
    """
    scores = []
    
    for doc in docs:
        if method == "keyword":
            score = calculate_relevance_keyword(query, doc)
        scores.append(score)
    
    # 최대값 기반 평가 (Top-1 문서의 품질로 판단)
    max_score = max(scores) if scores else 0.0
    
    # 액션 결정 (임계값 기반)
    if max_score >= 0.5:  # 50% 이상 겹침
        action = "correct"
    elif max_score < 0.2:  # 20% 미만 겹침
        action = "incorrect"
    else:  # 20~50% 사이
        action = "ambiguous"
    
    return max_score, action, scores

print("✅ Retrieval Evaluator 준비 완료!")
print()
print("📊 액션 결정 기준:")
print("   - score ≥ 0.5: correct (검색 성공) ✅")
print("   - score < 0.2: incorrect (검색 실패) ❌")
print("   - 0.2 ≤ score < 0.5: ambiguous (애매함) ⚠️")
print()
print("💡 평가 방식: 최대값 기반 (Top-1 문서 품질로 판단)")

In [ ]:
# Evaluator 테스트

test_cases = [
    {
        "query": "공서양속 위반 계약",
        "expected_action": "correct",
        "reason": "민법 103조와 매우 관련 높음"
    },
    {
        "query": "오늘 날씨가 어때요?",
        "expected_action": "incorrect",
        "reason": "법률 문서와 전혀 관련 없음"
    },
    {
        "query": "계약 해지 방법",
        "expected_action": "ambiguous",
        "reason": "일부만 관련 (3개 토큰 중 1개 매칭)"
    }
]

print("🧪 Retrieval Evaluator 테스트")
print("=" * 70)
print()

for i, case in enumerate(test_cases, 1):
    query = case["query"]
    expected = case["expected_action"]
    reason = case["reason"]
    
    print(f"테스트 {i}: {query}")
    print(f"   기대: {expected} ({reason})")
    
    # 검색
    docs = vectorstore.similarity_search(query, k=3)
    
    # 평가
    max_score, action, individual_scores = evaluate_retrieval(query, docs)
    
    print(f"   결과: {action} (최고 점수: {max_score:.3f})")
    
    # 개별 문서 점수
    print(f"   문서별 점수:")
    for j, (doc, score) in enumerate(zip(docs, individual_scores), 1):
        law = doc.metadata.get('law', 'unknown')
        print(f"      {j}. {law}: {score:.3f}")
    
    # 결과 체크
    if action == expected:
        print(f"   ✅ 성공! 예상대로 '{action}' 판정")
    else:
        print(f"   ⚠️ 예상({expected})과 다름({action})")
    
    print()

print("=" * 70)
print("💡 Evaluator가 검색 결과의 품질을 자동으로 판단합니다!")

## 🚦 Part 2: 3가지 액션 경로

### Corrective RAG의 핵심
**평가 결과에 따라 다른 전략 실행!**

```
Query → 검색 → 평가 → 액션 선택
                      ↓
        ┌─────────────┼─────────────┐
        ↓             ↓             ↓
    Correct      Ambiguous     Incorrect
    (0.5+)       (0.2~0.5)      (0.2-)
        ↓             ↓             ↓
    Reranking    검색+웹결합    웹검색폴백
```

### 액션 1: Correct (검색 성공)
- 검색 결과를 신뢰할 수 있음
- 05번에서 배운 **Reranking** 적용
- 최고 품질의 답변 생성

### 액션 2: Incorrect (검색 실패)
- 검색 결과가 전혀 관련 없음
- **폴백 전략** 실행:
  - Option 1: 웹 검색
  - Option 2: LLM 자체 지식
  - Option 3: "답변 불가" 반환

### 액션 3: Ambiguous (애매함)
- 검색 결과가 부분적으로만 관련
- **검색 + 웹 결합** 전략
- 더 많은 소스로 보완

### 실습: 3가지 액션 구현

In [ ]:
# 액션 1: Correct (검색 성공)

def action_correct(query, k=3):
    """
    Correct 액션: 검색 결과를 신뢰하고 그대로 사용
    """
    print("✅ ACTION: CORRECT (검색 성공)")
    print("   전략: 검색 결과 신뢰")
    print()
    
    # 벡터 검색
    results = vectorstore.similarity_search(query, k=k)
    
    print(f"   검색 결과 (Top-{k}):")
    for i, doc in enumerate(results, 1):
        law = doc.metadata.get('law', 'unknown')
        print(f"      {i}. {law}")
    
    return results

# 테스트: Correct 액션
print("🧪 액션 1 테스트: Correct")
print("=" * 60)
print()

query = "공서양속 위반 계약"
print(f"❓ 질문: {query}")
print()

# 1단계: 검색
docs = vectorstore.similarity_search(query, k=3)

# 2단계: 평가
max_score, action, _ = evaluate_retrieval(query, docs)
print(f"📊 평가 결과: {action} (점수: {max_score:.3f})")
print()

# 3단계: 액션 실행
if action == "correct":
    results = action_correct(query, k=3)
    print()
    print("💬 답변 생성 가능: 민법 제103조 내용으로 정확한 답변 제공 ✅")
else:
    print(f"   ⚠️ 예상과 다른 액션: {action}")

In [ ]:
# 액션 2: Incorrect (검색 실패 - 폴백 전략)

def action_incorrect_fallback(query):
    """
    Incorrect 액션: 검색 실패 시 폴백
    
    폴백 옵션:
    1. 웹 검색 (실제 구현 시)
    2. LLM 자체 지식 (구현 가능)
    3. "답변 불가" 메시지 (안전)
    """
    print("❌ ACTION: INCORRECT (검색 실패)")
    print("   전략: 폴백 실행")
    print()
    
    # 폴백 전략 선택
    print("   🌐 폴백 옵션:")
    print("      1. 웹 검색 (Google, Naver 등)")
    print("      2. LLM 자체 지식 활용")
    print("      3. '답변 불가' 메시지 반환")
    print()
    
    # 이 실습에서는 Option 3 (안전한 방법)
    print("   선택된 폴백: Option 3 (답변 불가)")
    print()
    
    fallback_message = f"""
죄송합니다. '{query}'에 대한 정보가 법률 문서 데이터베이스에 없습니다.

💡 도움말:
- 법률 관련 질문인 경우: 조문 번호나 법률 용어를 포함해주세요.
- 일반 질문인 경우: 다른 검색 엔진을 이용해주세요.
"""
    
    print("   💬 폴백 메시지:")
    print(fallback_message)
    
    return fallback_message

# 테스트: Incorrect 액션
print("🧪 액션 2 테스트: Incorrect")
print("=" * 60)
print()

query = "오늘 날씨가 어때요?"
print(f"❓ 질문: {query}")
print()

# 1단계: 검색
docs = vectorstore.similarity_search(query, k=3)

# 2단계: 평가
max_score, action, _ = evaluate_retrieval(query, docs)
print(f"📊 평가 결과: {action} (점수: {max_score:.3f})")
print()

# 3단계: 액션 실행
if action == "incorrect":
    fallback_message = action_incorrect_fallback(query)
    print("✅ 폴백 전략으로 안전하게 처리됨!")
else:
    print(f"   ⚠️ 예상과 다른 액션: {action}")

print()
print("💡 핵심: 검색 실패 시 엉뚱한 답변보다 폴백이 안전!")

In [ ]:
# 액션 3: Ambiguous (애매함 - 검색 + 보강)

def action_ambiguous_combine(query, k=3):
    """
    Ambiguous 액션: 검색 결과를 보강
    
    전략:
    1. 기존 검색 결과 유지
    2. 추가 소스로 보강 (웹 검색, 다른 DB 등)
    3. 결합된 결과 반환
    """
    print("⚠️ ACTION: AMBIGUOUS (애매함)")
    print("   전략: 검색 결과 + 추가 소스 보강")
    print()
    
    # 1단계: 기존 검색
    retrieval_docs = vectorstore.similarity_search(query, k=k)
    
    print(f"   기존 검색 결과 (Top-{k}):")
    for i, doc in enumerate(retrieval_docs, 1):
        law = doc.metadata.get('law', 'unknown')
        print(f"      {i}. {law}")
    
    print()
    print("   🌐 추가 소스 보강:")
    
    # 2단계: 결합 - 웹 문서 추가 (시뮬레이션)
    web_doc = Document(
        page_content="계약 해지는 당사자 간 합의 또는 법정 해지 사유(채무불이행, 해제권 행사 등)로 가능합니다. 민법 제543조 참조.",
        metadata={"law": "웹 검색 결과", "category": "웹"}
    )
    print("      - 웹 검색 결과 추가 ✅")
    
    combined_docs = retrieval_docs + [web_doc]
    
    print()
    print("   최종 결과 (보강 후):")
    for i, doc in enumerate(combined_docs, 1):
        law = doc.metadata.get('law', 'unknown')
        print(f"      {i}. {law}")
    
    return combined_docs

# 테스트: Ambiguous 액션
print("🧪 액션 3 테스트: Ambiguous")
print("=" * 60)
print()

query = "계약 해지 방법"
print(f"❓ 질문: {query}")
print()

# 1단계: 검색
docs = vectorstore.similarity_search(query, k=3)

# 2단계: 평가
max_score, action, _ = evaluate_retrieval(query, docs)
print(f"📊 평가 결과: {action} (점수: {max_score:.3f})")
print()

# 3단계: 액션 실행
if action == "ambiguous":
    results = action_ambiguous_combine(query, k=3)
    print()
    print("✅ 보강 전략으로 더 풍부한 답변 가능!")
elif action == "correct":
    print("   💡 예상보다 좋은 결과 → Correct 액션으로 진행")
    results = action_correct(query, k=3)
else:
    print(f"   ⚠️ 예상과 다른 액션: {action}")

print()
print("💡 핵심: 애매할 때는 추가 소스로 보강!")

---

## 🎯 퀴즈: 액션 예측하기

3가지 액션을 배웠습니다! 이제 질문을 보고 어떤 액션이 나올지 예측해보세요.

**질문 1**: "공서양속 위반 계약 무효"  
**질문 2**: "오늘 점심 메뉴 추천해줘"  
**질문 3**: "계약 위반 손해배상"

각 질문은 어떤 액션이 나올까요?
- **Correct** (score ≥ 0.5): 검색 성공
- **Incorrect** (score < 0.2): 검색 실패
- **Ambiguous** (0.2 ~ 0.5): 애매함

아래 셀에서 실행해서 확인하세요!

In [ ]:
# 3가지 질문으로 액션 예측 퀴즈

quiz_questions = [
    {"query": "상해죄 성립 요건", "hint": "명확한 법률 키워드"},
    {"query": "비트코인 투자 방법 알려줘", "hint": "법률과 완전 무관"},
    {"query": "회사 설립 절차", "hint": "일반적인 법률 용어"}
]

print("🎯 액션 예측 퀴즈")
print("=" * 60)
print()

for i, item in enumerate(quiz_questions, 1):
    query = item["query"]
    hint = item["hint"]
    
    print(f"질문 {i}: '{query}'")
    print(f"   힌트: {hint}")
    
    # 검색
    docs = vectorstore.similarity_search(query, k=3)
    
    # 평가
    max_score, action, _ = evaluate_retrieval(query, docs)
    
    print(f"   결과: {action.upper()} (점수: {max_score:.3f})")
    
    # 설명
    if action == "correct":
        print(f"   ✅ 검색 성공! 법률 키워드가 잘 매칭됨")
    elif action == "incorrect":
        print(f"   ❌ 검색 실패! 법률 문서와 무관함")
    else:
        print(f"   ⚠️ 애매함! 일반적인 법률 용어")
    
    print()

<details>
<summary>📝 정답 보기 (클릭)</summary>

**정답:**

1. **"상해죄 성립 요건"** → **Correct** ✅
   - 형법 제257조와 직접 관련
   - 키워드: 상해, 신체 (정확 매칭)
   - score ≥ 0.5

2. **"비트코인 투자 방법 알려줘"** → **Incorrect** ❌
   - 법률 문서와 전혀 무관
   - 키워드 겹침 거의 없음
   - score < 0.2
   - → 폴백 전략 실행!

3. **"회사 설립 절차"** → **Ambiguous** ⚠️
   - 상법과 부분적으로 관련
   - 일반적인 법률 용어 (특정 조문 아님)
   - 0.2 ≤ score < 0.5
   - → 검색 + 보강 전략!

**핵심:**
- CRAG는 점수(score)로 검색 품질을 평가
- 품질에 따라 다른 전략 실행
- 견고한 시스템 구축!

</details>

In [ ]:
# 종합 CRAG 파이프라인

def corrective_rag(query, k=3, verbose=True):
    """
    완전한 Corrective RAG 파이프라인
    
    Steps:
    1. Retrieval (검색)
    2. Evaluation (평가)
    3. Action Selection (액션 선택)
    4. Answer Generation (답변 생성)
    """
    if verbose:
        print("🎯 Corrective RAG 파이프라인 실행")
        print("=" * 60)
        print(f"Query: {query}")
        print()
    
    # Step 1: Retrieval
    if verbose:
        print("📚 Step 1: Retrieval (검색)")
    docs = vectorstore.similarity_search(query, k=k)
    if verbose:
        print(f"   검색 완료: {len(docs)}개 문서")
        print()
    
    # Step 2: Evaluation
    if verbose:
        print("📊 Step 2: Evaluation (평가)")
    max_score, action, individual_scores = evaluate_retrieval(query, docs)
    if verbose:
        print(f"   평가 점수: {max_score:.3f}")
        print(f"   결정된 액션: {action.upper()}")
        print()
    
    # Step 3: Action Selection
    if verbose:
        print("🚦 Step 3: Action Selection (액션 실행)")
    
    result = {
        "query": query,
        "eval_score": max_score,
        "action": action,
        "docs": None,
        "answer": None
    }
    
    if action == "correct":
        if verbose:
            print("   ✅ Correct → 검색 결과 신뢰")
        final_docs = action_correct(query, k=k)
        result["docs"] = final_docs
        result["answer"] = f"검색 성공! Top 문서: {final_docs[0].metadata['law']}"
        
    elif action == "incorrect":
        if verbose:
            print("   ❌ Incorrect → 폴백 전략")
        result["answer"] = f"죄송합니다. '{query}'에 대한 법률 정보가 없습니다. 다른 질문을 시도해주세요."
        
    else:  # ambiguous
        if verbose:
            print("   ⚠️ Ambiguous → 검색 + 보강")
        combined_docs = action_ambiguous_combine(query, k=k)
        result["docs"] = combined_docs
        result["answer"] = f"부분적 매칭. 관련 문서: {', '.join([d.metadata['law'] for d in combined_docs[:2]])}"
    
    if verbose:
        print()
        print("💬 Step 4: Answer Generation (답변)")
        print(f"   {result['answer']}")
        print()
    
    return result

print("✅ 종합 CRAG 파이프라인 준비 완료!")

In [ ]:
# 법률 도메인 종합 테스트

comprehensive_test_cases = [
    {
        "query": "폭행 상해 기준",
        "expected_action": "correct",
        "description": "명확한 법률 질문"
    },
    {
        "query": "재물 절취 처벌",
        "expected_action": "correct",
        "description": "특정 조문 관련"
    },
    {
        "query": "법률 자문 비용",
        "expected_action": "ambiguous",
        "description": "일반적 법률 용어"
    },
    {
        "query": "맛집 추천해줘",
        "expected_action": "incorrect",
        "description": "법률과 무관한 질문"
    },
    {
        "query": "축구 경기 결과",
        "expected_action": "incorrect",
        "description": "완전히 다른 도메인"
    }
]

print("🧪 법률 도메인 종합 테스트")
print("=" * 70)
print()

results_table = []

for i, case in enumerate(comprehensive_test_cases, 1):
    query = case["query"]
    expected = case["expected_action"]
    description = case["description"]
    
    print(f"\n### 테스트 {i}: {description}")
    print(f"Query: \"{query}\"")
    print(f"기대 액션: {expected}")
    print()
    
    # CRAG 실행
    result = corrective_rag(query, k=3, verbose=False)
    
    # 결과 출력
    actual_action = result["action"]
    eval_score = result["eval_score"]
    
    print(f"   평가 점수: {eval_score:.3f}")
    print(f"   실제 액션: {actual_action}")
    print(f"   답변: {result['answer'][:80]}..." if len(result['answer']) > 80 else f"   답변: {result['answer']}")
    
    # 성공 여부
    success = "✅" if actual_action == expected else "⚠️"
    print(f"   {success} {'성공' if actual_action == expected else f'예상({expected})과 다름'}")
    
    # 테이블용 데이터
    results_table.append({
        "질문": query[:20] + "..." if len(query) > 20 else query,
        "유형": description,
        "점수": f"{eval_score:.3f}",
        "액션": actual_action,
        "성공": success
    })

print("\n" + "=" * 70)
print("\n📊 종합 결과 테이블:\n")
df = pd.DataFrame(results_table)
print(df.to_string(index=False))

# 성공률 계산
success_count = sum(1 for r in results_table if r["성공"] == "✅")
total_count = len(results_table)
success_rate = success_count / total_count * 100

print(f"\n🎯 전체 성공률: {success_count}/{total_count} ({success_rate:.0f}%)")
print("\n💡 CRAG가 다양한 질문 유형을 적절히 처리합니다!")

---

## 🎛️ 실습: 임계값 튜닝

### 임계값이란?

CRAG에서 **액션을 결정하는 기준 점수**

```python
if score >= 0.5:  # correct_threshold
    action = "correct"
elif score < 0.2:  # incorrect_threshold
    action = "incorrect"
else:
    action = "ambiguous"
```

### 임계값에 따른 효과

- **correct_threshold ↑ (0.6~0.7)**: Correct 판정 엄격 → Ambiguous 증가
- **correct_threshold ↓ (0.3~0.4)**: Correct 판정 관대 → Correct 증가
- **incorrect_threshold ↑ (0.3)**: Incorrect 판정 엄격 → Ambiguous 증가
- **incorrect_threshold ↓ (0.1)**: Incorrect 판정 관대 → Incorrect 증가

### 실습: 임계값을 바꿔가며 액션 변화 확인!

현재 법률 문서에서 최적의 임계값은?

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 임계값을 변경하면서 액션이 어떻게 바뀌는지 확인하세요

correct_threshold = 0.5  # 0.3 ~ 0.7 사이 값으로 변경
incorrect_threshold = 0.2  # 0.1 ~ 0.3 사이 값으로 변경

print(f"🎛️ 임계값 설정")
print(f"   Correct: score ≥ {correct_threshold}")
print(f"   Incorrect: score < {incorrect_threshold}")
print(f"   Ambiguous: {incorrect_threshold} ≤ score < {correct_threshold}")
print("=" * 60)
print()

# 수정된 평가 함수
def evaluate_with_custom_thresholds(query, docs, correct_th, incorrect_th):
    scores = [calculate_relevance_keyword(query, doc) for doc in docs]
    max_score = max(scores) if scores else 0.0
    
    if max_score >= correct_th:
        action = "correct"
    elif max_score < incorrect_th:
        action = "incorrect"
    else:
        action = "ambiguous"
    
    return max_score, action

# 테스트 케이스
test_queries = [
    "공서양속 위반",
    "계약 해지",
    "오늘 날씨"
]

action_counts = {"correct": 0, "incorrect": 0, "ambiguous": 0}

for query in test_queries:
    docs = vectorstore.similarity_search(query, k=3)
    score, action = evaluate_with_custom_thresholds(query, docs, correct_threshold, incorrect_threshold)
    
    action_counts[action] += 1
    
    print(f"'{query[:15]}...' → {action.upper()} (점수: {score:.3f})")

print()
print("=" * 60)
print(f"📊 액션 분포:")
print(f"   Correct: {action_counts['correct']}/3")
print(f"   Incorrect: {action_counts['incorrect']}/3")
print(f"   Ambiguous: {action_counts['ambiguous']}/3")
print()
print("💡 실험해보세요:")
print("   - correct_threshold=0.6: Correct 엄격 → Ambiguous 증가")
print("   - correct_threshold=0.4: Correct 관대 → Correct 증가")
print("   - incorrect_threshold=0.3: Incorrect 엄격 → Ambiguous 증가")

<details>
<summary>📝 정답 예시 (클릭)</summary>

**실험 결과 예시:**

### 기본 설정 (0.5 / 0.2):
```
절도죄 형량 → CORRECT (0.667)
상호 등록 방법 → AMBIGUOUS (0.333)
주식 투자 팁 → INCORRECT (0.000)
```
→ 균형잡힌 분류

### Correct 엄격 (0.7 / 0.2):
```
절도죄 형량 → AMBIGUOUS (0.667)  ← 0.7 미만이라 Ambiguous
상호 등록 방법 → AMBIGUOUS (0.333)
주식 투자 팁 → INCORRECT (0.000)
```
→ Ambiguous 증가

### Correct 관대 (0.4 / 0.2):
```
절도죄 형량 → CORRECT (0.667)
상호 등록 방법 → AMBIGUOUS (0.333)  ← 여전히 0.4 미만
주식 투자 팁 → INCORRECT (0.000)
```
→ Correct 증가

**핵심:**
- 임계값은 도메인 특성에 맞게 조정
- 너무 엄격하면 Ambiguous 폭증
- 너무 관대하면 오답 위험

</details>

## 📊 Part 4: Naive RAG vs CRAG 성능 비교

### 비교 포인트

| 측면 | Naive RAG | Corrective RAG (CRAG) |
|------|-----------|----------------------|
| **검증** | 없음 | Retrieval Evaluator |
| **실패 처리** | 엉뚱한 답변 | 폴백 전략 |
| **견고성** | 낮음 | 높음 |
| **사용자 경험** | 혼란 | 명확 |

### 핵심 차이

**Naive RAG:**
```
Query: "오늘 날씨 어때?"
  → 검색: 법률 문서 3개
  → 답변: "민법 제1조에 따르면..." ❌ (엉뚱함)
```

**CRAG:**
```
Query: "오늘 날씨 어때?"
  → 검색: 법률 문서 3개
  → 평가: 점수 0.05 → Incorrect
  → 폴백: "죄송합니다. 법률 정보가 아닙니다." ✅ (안전)
```

### 실습: 직접 비교

In [ ]:
# Naive RAG vs CRAG 비교

def naive_rag(query, k=3):
    """Naive RAG: 검증 없이 바로 검색 결과 사용"""
    docs = vectorstore.similarity_search(query, k=k)
    
    # 무조건 첫 번째 문서 기반 답변
    if docs:
        top_doc = docs[0]
        law = top_doc.metadata.get('law', 'unknown')
        return f"{law} 기반 답변: {top_doc.page_content[:100]}..."
    else:
        return "문서를 찾을 수 없습니다."

# 비교 테스트 케이스
comparison_cases = [
    {
        "query": "공서양속 위반 계약",
        "type": "적절한 질문"
    },
    {
        "query": "오늘 날씨가 어때요?",
        "type": "부적절한 질문"
    },
    {
        "query": "점심 메뉴 추천해줘",
        "type": "완전히 무관한 질문"
    }
]

print("🔍 Naive RAG vs CRAG 직접 비교")
print("=" * 80)
print()

comparison_results = []

for i, case in enumerate(comparison_cases, 1):
    query = case["query"]
    qtype = case["type"]
    
    print(f"\n### 케이스 {i}: {qtype}")
    print(f"Query: \"{query}\"")
    print()
    
    # Naive RAG
    print("❌ Naive RAG:")
    naive_answer = naive_rag(query, k=3)
    print(f"   {naive_answer[:80]}..." if len(naive_answer) > 80 else f"   {naive_answer}")
    
    # 검증 없이 무조건 답변 → 부적절한 경우 문제
    naive_ok = "✅" if qtype == "적절한 질문" else "❌"
    print(f"   평가: {naive_ok} {'적절' if naive_ok == '✅' else '부적절 (엉뚱한 답변)'}")
    
    print()
    
    # CRAG
    print("✅ CRAG:")
    crag_result = corrective_rag(query, k=3, verbose=False)
    crag_answer = crag_result["answer"]
    crag_action = crag_result["action"]
    print(f"   액션: {crag_action}")
    print(f"   {crag_answer[:80]}..." if len(crag_answer) > 80 else f"   {crag_answer}")
    
    # CRAG는 항상 적절하게 처리
    crag_ok = "✅"
    print(f"   평가: {crag_ok} 적절 (검증 후 답변)")
    
    # 결과 저장
    comparison_results.append({
        "질문 유형": qtype,
        "Naive RAG": naive_ok,
        "CRAG": crag_ok
    })
    
    print()
    print("-" * 80)

print("\n📊 비교 결과 요약:\n")
df_comparison = pd.DataFrame(comparison_results)
print(df_comparison.to_string(index=False))

# 성공률
naive_success = sum(1 for r in comparison_results if r["Naive RAG"] == "✅")
crag_success = sum(1 for r in comparison_results if r["CRAG"] == "✅")
total = len(comparison_results)

print(f"\n🎯 성공률:")
print(f"   Naive RAG: {naive_success}/{total} ({naive_success/total*100:.0f}%)")
print(f"   CRAG: {crag_success}/{total} ({crag_success/total*100:.0f}%)")
print()
print("💡 CRAG의 핵심 가치: 부적절한 질문에 대한 견고성!")

In [ ]:
import matplotlib.pyplot as plt

# 데이터 준비
categories = ['Relevant\nQuestion', 'Irrelevant\nQuestion', 'Off-topic\nQuestion']
naive_scores = [1, 0, 0]  # Naive RAG: Only handles relevant questions well
crag_scores = [1, 1, 1]   # CRAG: Handles all cases robustly

x = range(len(categories))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar([i - width/2 for i in x], naive_scores, width, 
               label='Naive RAG', color='red', alpha=0.7)
bars2 = ax.bar([i + width/2 for i in x], crag_scores, width,
               label='CRAG', color='green', alpha=0.7)

ax.set_ylabel('Appropriateness (1=Appropriate, 0=Inappropriate)', fontsize=12)
ax.set_title('Naive RAG vs CRAG: Robustness Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=11)
ax.set_ylim(0, 1.3)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# 막대 위에 라벨
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        label = '✅' if height == 1 else '❌'
        ax.text(bar.get_x() + bar.get_width()/2, height + 0.05,
                label, ha='center', fontsize=16)

plt.tight_layout()
plt.show()

# ---- 해석 한글로 ----
print("📊 시각화 해석:")
print()
print("❌ Naive RAG:")
print("   - 적절한 질문: ✅ (성공)")
print("   - 부적절한 질문: ❌ (실패 - 엉뚱한 답변)")
print("   - 무관한 질문: ❌ (실패 - 혼란스러운 답변)")
print()
print("✅ CRAG:")
print("   - 적절한 질문: ✅ (정확한 처리)")
print("   - 부적절한 질문: ✅ (폴백 처리)")
print("   - 무관한 질문: ✅ (안전하게 무시 또는 처리)")
print()
print("💡 핵심: CRAG는 모든 질문 유형에서 안정적으로 동작하는 견고한 시스템입니다!")


## 🎓 핵심 정리

### Corrective RAG (CRAG)란?

**검색 결과를 평가하고 품질에 따라 다른 전략 실행!**

```
Query → 검색 → 평가 → 액션 선택
              ↓
    ┌─────────┼─────────┐
    ↓         ↓         ↓
 Correct  Ambiguous  Incorrect
   ↓         ↓         ↓
 Rerank   보강      폴백
```

### 3가지 핵심 요소

**1. Retrieval Evaluator**
- 검색 결과의 relevance 점수 계산 (0.0~1.0)
- 키워드 기반 또는 LLM 기반

**2. 3가지 액션**
- **Correct** (≥0.5): Reranking
- **Incorrect** (<0.2): 폴백 (웹 검색/답변 불가)
- **Ambiguous** (0.2~0.5): 검색 + 보강

**3. 폴백 전략**
- 웹 검색 / LLM 지식 / "답변 불가" 메시지

### 언제 사용?
- ✅ 다양한 질문 유형 (도메인 내/외 섞임)
- ✅ 높은 신뢰도 요구 (엉뚱한 답변 방지)
- ✅ 프로덕션 환경 (견고성 중요)

### Naive vs CRAG

| 측면 | Naive RAG | CRAG |
|------|-----------|------|
| 검증 | 없음 | Evaluator |
| 실패 처리 | 엉뚱한 답변 | 폴백 |
| 견고성 | 낮음 | 높음 |

---

**🎉 Day 2 완료!**
01-06번: Naive → Multi-Query → Metadata → Hybrid → CRAG ✅